In [1]:
from PyQt6.QtWidgets import *
from PyQt6.QtCore import *
from PyQt6.QtGui import *
import sys

### Classe tableau pour la construction de courbe (copy/paste enabled)

In [2]:
class TableWidget(QTableWidget):

    def __init__(self, rows =26 , columns = 2, parent=None): #default size is without jibar
        super().__init__(rows, columns)
        

        self.setHorizontalHeaderLabels(["Product","Price"])
 

    def keyPressEvent(self, event):  # copy paste in table
        if event.key() == Qt.Key.Key_V and (event.modifiers() & Qt.KeyboardModifier.ControlModifier):
            selection = self.selectedIndexes()
            if selection:
                row_anchor = selection[0].row()
                column_anchor = selection[0].column()
                clipboard = QApplication.clipboard()

                rows = clipboard.text().split('\n')
                for indx_row, row in enumerate(rows):
                    values = row.split('\t')
                    for indx_col, value in enumerate(values):
                        item = QTableWidgetItem(value)
                        self.setItem(row_anchor + indx_row, column_anchor + indx_col, item)
            super().keyPressEvent(event)

In [3]:
class PayFreqbox(QComboBox):
    def __init__(self ):
        super(PayFreqbox,self).__init__()
        
        
        self.addItems(["Once","Annual","SemiAnnual","EveryFourthMonth","Quarterly","Bimonthly",
                       "Monthly","Weekly","Daily"])
        self.setEditable(False)
        self.currentIndexChanged.connect(self.selectionchange)
        
    def selectionchange(self,i):
        print(self.currentText())

        

### Combo box pour la convention BADDAys

In [4]:
class BusAdjbox(QComboBox):
    def __init__(self ):
        super(BusAdjbox,self).__init__()
        
        
        self.addItems(["Following","Modified Following","Preceding", "Modified Preceding", "Unadjusted"])
        self.setEditable(False)
        self.currentIndexChanged.connect(self.selectionchange)
        #self.setLayout(QHBoxLayout())
    def selectionchange(self,i):
        print(self.currentText())

        

### Combo box pour la convention comptage de j

In [5]:
class DCountbox(QComboBox):
    def __init__(self ):
        super(DCountbox,self).__init__()

        
        self.addItems(["SimpleDayCounter", "Thirty360 30/360 ",
                            "Actual/360", "Actual/365(Fixed)",
                           "Actuel/Actual", "Business / 252"
                           ])
        self.setEditable(False)
        self.currentIndexChanged.connect(self.selectionchange)
        self.setLayout(QHBoxLayout())


    def selectionchange(self,i):
        print(self.currentText())

        

In [6]:
class Maturity(QHBoxLayout):       
        def __init__(self):
                super(Maturity,self).__init__()
                interval=QLineEdit()
                interval.setFixedWidth(30)
                self.addWidget(interval)
                self.addWidget(QDateEdit())
        




In [7]:
class MainLayout(QGridLayout):
            
     def __init__(self):

        super(MainLayout,self).__init__()   
        # General Parameters
        self.addWidget(QLabel("Calendar") ,0,0 )
        self.addWidget(QLabel("South Africa") ,0,1 )
        self.addWidget(QLabel("Fixing Days") ,1,0 )
        self.addWidget(QSpinBox() ,1,1 )
        self.addWidget(QLabel("Settlement Date") ,2,0 )
        self.addWidget(QDateEdit(QDate.currentDate()) ,2,1 )
        self.addWidget(QLabel("EOM") ,3,0 )
        self.addWidget(QCheckBox() ,3,1 )

        lab = QLabel()
        pixmap = QPixmap('flag.png')
        lab.resize(pixmap.width(),pixmap.height())
        lab.setPixmap(pixmap)

        self.addWidget(lab,0,3,2,2)

        update_curve_button = QPushButton("Curve Construction")
        update_curve_button.setSizePolicy(QSizePolicy.Policy.Expanding,QSizePolicy.Policy.Expanding)
        self.addWidget(update_curve_button,2,3,2,2)


        self.addWidget(QLabel("Nominal") ,7,1,alignment= Qt.AlignmentFlag(4) )
        self.addWidget(QSpinBox() ,7,3 )


        #Fixed leg parameters


        self.addWidget(QLabel("Fixed") ,5,0 , alignment= Qt.AlignmentFlag(2) )
        self.addWidget(QLabel("Leg") ,5,1 )

        self.addWidget(QLabel("Effective Date") ,9,0 )
        self.addWidget(QDateEdit() ,9,1)

        self.addWidget(QLabel("Maturity (e.g.4Y,3M)") ,10,0 )
        self.addLayout(Maturity() ,10,1 )

        self.addWidget(QLabel("Fixed Rate") ,11,0 )
        self.addWidget(QDoubleSpinBox() ,11,1 )

        self.addWidget(QLabel("Payment Freq") ,12,0 )
        self.addWidget(PayFreqbox() ,12,1 )

        self.addWidget(QLabel("Day Count") ,13,0 )
        self.addWidget(DCountbox() ,13,1 )

        self.addWidget(QLabel("Bus adjustment") ,14,0 )
        self.addWidget(BusAdjbox() ,14,1 )
        


        #Floating leg parameter

        self.addWidget(QLabel("Floating") ,5,3 , alignment= Qt.AlignmentFlag(2) )
        self.addWidget(QLabel("Leg") ,5,4 )

        self.addWidget(QLabel("Effective Date") ,9,3 )
        self.addWidget(QDateEdit() ,9,4)

        self.addWidget(QLabel("Maturity (e.g. 4Y,3M)") ,10,3)
        self.addLayout(Maturity() ,10,4 )

        self.addWidget(QLabel("Index") ,11,3 )
        self.addWidget(QLabel("3M-Jibar") ,11,4)

        self.addWidget(QLabel("Spread") ,12,3 )
        self.addWidget(QDoubleSpinBox() ,12,4 )

        self.addWidget(QLabel("Last index fix") ,13,3 )

        self.addWidget(QDoubleSpinBox(),13,4)
        # layout.addWidget(QLineEdit() ,13,4 )

        self.addWidget(QLabel("Pmt Freq") ,14,3)
        self.addWidget(PayFreqbox() ,14,4 )

        self.addWidget(QLabel("Day Count") ,15,3)
        self.addWidget(DCountbox() ,15,4 )

        self.addWidget(QLabel("BusDay Adj") ,16,3)
        self.addWidget(BusAdjbox() ,16,4 )

In [ ]:
class CurveWidget(QHBoxLayout):
            
     def __init__(self):

        super(CurveWidget,self).__init__()  
        self.addWidget(TableWidget())
        self.addWidget() 

In [8]:
class MainWindow(QMainWindow):

    def __init__(self):
        super(MainWindow, self).__init__()

        self.setWindowTitle("ZAR Vanilla Swap Pricer")
        self.setFixedSize(QSize(500,520))

        tabs = QTabWidget()
        tabs.setTabPosition(QTabWidget.TabPosition.South)
        tabs.setMovable(False)               

        
        mainwidget = QWidget()
        mainwidget.setLayout(MainLayout())

        curvewidget = QWidget()
        curvewidget.setLayout(CurveWidget())


        tabs.addTab(mainwidget, "main")
        tabs.addTab(curvewidget, "red")
        

        self.setCentralWidget(tabs)

        

app = QApplication(sys.argv)

window = MainWindow()
window.show()

app.exec()



        

0